In [166]:
import requests
import pandas
import base64
from datetime import datetime,timedelta
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
import json

In [170]:
#Connect to MySQL DB

connection = mysql.connector.connect(host = "localhost", database = "jason", user = "root", password = "q2_M4yEX")

In [2]:
class SpotifyAPI(object):
    access_token = ''
    access_token_expires = ''
    client_id = ''
    client_secret = ''
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_token_header(self):
        #returns a base64 string (not bytes)
        if self.client_id == '' or self.client_secret == '':
            raise Exception("client_id or client_secret has not been updated")
        client_creds = "%s:%s" % (self.client_id, self.client_secret)
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return {
        "Authorization" : f"Basic {client_creds_b64.decode()}"
}
    
    
    def get_token_data(self):
        return {
        "grant_type" : "client_credentials"
        }
    
    def perform_auth(self):
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        token_url = self.token_url
        
        r = requests.post(token_url, data = token_data, headers = token_header)
        valid_request = r.status_code in range(200,299) #This is to check if status_code is valid
        if valid_request:
            token_response_data = r.json() #r.json() returns a dictionary
            now = datetime.now()
            access_token = token_response_data['access_token']
            expires_in = token_response_data['expires_in']
            expires = now + timedelta(seconds=expires_in)
            self.access_token_expires = expires
            self.access_token = access_token
            return "Success"
        return "Failed"

In [3]:
client_id = '135ab778e09d46299134e28364c49f34'
client_secret = '808207dc7a8148fc88deb1d550a3bf8d'

client_creds = "%s:%s" % (client_id,client_secret)
client_creds_test = f"{client_id}:{client_secret}"
print(client_creds)
print(client_creds_test)

#To change this string to base64 encoded string
client_creds_b64 = base64.b64encode(client_creds.encode())
print(client_creds_b64)

135ab778e09d46299134e28364c49f34:808207dc7a8148fc88deb1d550a3bf8d
135ab778e09d46299134e28364c49f34:808207dc7a8148fc88deb1d550a3bf8d
b'MTM1YWI3NzhlMDlkNDYyOTkxMzRlMjgzNjRjNDlmMzQ6ODA4MjA3ZGM3YTgxNDhmYzg4ZGViMWQ1NTBhM2JmOGQ='


In [4]:
# do a lookup for a token, this token is for future requests
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = { 
        "grant_type" : "client_credentials"
}
token_headers = {
        "Authorization" : f"Basic {client_creds_b64.decode()}"
}

In [5]:
spotify = SpotifyAPI(client_id, client_secret)

print(spotify.perform_auth())
access_token = spotify.access_token

Success


In [22]:
#Use Access Token to access Spotify API
print(access_token)
print(spotify.access_token_expires)


header = {
    "Authorization" : f"Bearer {access_token}"
}

data = {
    "q" : "Bang Dream" ,
    "type" : "track" ,
    "limit" : 50
}

#endpoint has to look like this "https://api.spotify.com/v1/search?q=tania%20bowra&type=artist"
#encode spaces with "%20" or "+"
endpoint = f"https://api.spotify.com/v1/search?q={data['q'].replace(' ','%20').lower()}&type={data['type']}&limit={data['limit']}"
print(endpoint)

BQCcz4Wo8QDjX8jPechFml7j5nvhyAuCl_83ZpFPF0DYsfQtBY-Nt9lvzt4MXdbMCqprx_swaWzAl0nYkFw
2020-07-30 18:38:28.094189
https://api.spotify.com/v1/search?q=bang%20dream&type=track&limit=50


In [23]:
r = requests.get(endpoint, headers = header)

In [117]:
json = r.json()
track_json = json['tracks']
item_json = track_json['items']
print(track_json.keys())
print(len(track_json['items'])) #should be the same as track_json['limit']
print(track_json['items'][0].keys())

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
50
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


In [122]:
print(len(item_json))
print(item_json[0]['album'])

50
{'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'}, 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX', 'id': '3Nrfpe0tUJi4K4DXYWgMUX', 'name': 'BTS', 'type': 'artist', 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/25uiPmTg16RbhZWAqwLBy5'}, 'href': 'https://api.spotify.com/v1/artists/25uiPmTg16RbhZWAqwLBy5', 'id': '25uiPmTg16RbhZWAqwLBy5', 'name': 'Charli XCX', 'type': 'artist', 'uri': 'spotify:artist:25uiPmTg16RbhZWAqwLBy5'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'O

In [163]:
#print(item_json[0]['album'].keys())
#print(item_json[0].keys())
test = []

for i in range(len(item_json)):
    
    artist_name = ''
    for j in range(len(track_json['items'][i]['album']['artists'])):
        artist_name = artist_name + ',' + track_json['items'][i]['album']['artists'][j]['name']
    
    #print(artist_name)

    temp_tuple = (artist_name,
                  item_json[i]['id'],
                  item_json[i]['name'],
                  #item_json[i]['available_markets'],
                  item_json[i]['album']['id'],
                  item_json[i]['album']['name'],
                  item_json[i]['album']['album_type'],
                  item_json[i]['album']['release_date'],
                  item_json[i]['duration_ms'] , 
                  item_json[i]['explicit'],
                  item_json[i]['track_number'],
                  item_json[i]['type']
                 )
    
    test.append(temp_tuple)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'bool'>
<class 'int'>
<class 'str'>


In [171]:
#Insert data into MySQL 

insert_query = """INSERT INTO spotify_tracks (artist_name, track_id, track_name, album_id, album_name, album_type, release_date, duration_ms, explicit, track_number, track_type)    
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                """
insert_data = test

try: 
    cursor = connection.cursor()
    cursor.executemany(insert_query,insert_data)
    connection.commit()
    print(cursor.rowcount,"Record inserted successfully into spotify_tracks table")
except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

50 Record inserted successfully into spotify_tracks table
MySQL connection is closed
